In [1]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

import shutil

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime as dt

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_SQL_a_DataFrame
import funcion_parametrizacion_variables

<h3>Ejecución de Funciones</h3>

In [2]:
resultado = funcion_parametrizacion_variables.parametrizacion_variables()
consulta_tRendimientosDevueltosActualizacion = resultado[11]
renombrar_actividades = resultado[4]

df_tRendimientosDevueltosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosDevueltosActualizacion)

Inicio de la conexión
Conexión exitosa ...


<h3>Parametrización de Variables</h3>

In [3]:

fecha_actual = dt.now()
fecha_estandarizada = str(fecha_actual.strftime("%Y%m%d"))

# TODO: LLamada a la capa de Terrenos Final
terrenosXhito = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"
df_terrenos = pd.DataFrame.spatial.from_featureclass(terrenosXhito)

# TODO: Parametrización BD
BD_CONSOLIDADOLOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada"
NOMBRE_CAPA_DEVUELTOS = 'TERRENOS_RECONOCIMIENTO_TRAMITES_DEVUELTOS_BCGS'

# TODO: Actualización del Reporte de Seguimiento
DIRECTORIO_SALIDA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento"
NOMBRE_XLSX = fecha_estandarizada + "_RendimientoEquipoReconocimiento_BCGS.xlsx"
RUTA_SALIDA_XLSX = os.path.join(DIRECTORIO_SALIDA_XLSX, NOMBRE_XLSX)
NOMBRE_HOJA = 'Seguimiento_Rad_Devueltos'

In [4]:

# TODO: Se actualizan los códigos de la actividad
for columna in df_tRendimientosDevueltosActualizacion.columns:
    if columna == 'actividad_actual_tramite':
        for llave, valor in renombrar_actividades.items():
            df_tRendimientosDevueltosActualizacion.loc[df_tRendimientosDevueltosActualizacion[columna]==llave, columna] = valor

df_tRendimientosDevueltosActualizacion['pversion'] = df_tRendimientosDevueltosActualizacion['pversion'].astype(int)

In [5]:

# TODO: Unificación de los radicados devueltos con las geometrías de los terrenos
df_devueltosXterrenos = pd.merge(left=df_terrenos, 
        right=df_tRendimientosDevueltosActualizacion, 
        left_on="codigo", 
        right_on="npn",
        how="inner"
        )

estandarizacion_columnas_devuXterrenos = ['codigo',                                 
        'pversion',
        'radicado',
        'ficha',
        'componente_responsable',
        'actividad_actual_tramite',
        'con_plano_cartografico',
        'usuario_inserta_plano',
        'area_ha_cmt12',
        'id_ui',
        'nombre_ui',
        'meta_hito',
        'SHAPE'
        ]

df_devueltosXterrenos = df_devueltosXterrenos[estandarizacion_columnas_devuXterrenos]
df_devueltosXterrenos = df_devueltosXterrenos.rename(columns={'pversion':'radicado_padre', 'radicado':'radicado_hijo'})

In [6]:

# TODO: Se crea la capa geográfica de devueltos
df_devueltosXterrenos.spatial.to_featureclass(location=os.path.join(BD_CONSOLIDADOLOCAL,NOMBRE_CAPA_DEVUELTOS))
print(f"Se crea la capa {NOMBRE_CAPA_DEVUELTOS}")

Se crea la capa TERRENOS_RECONOCIMIENTO_TRAMITES_DEVUELTOS_BCGS


In [7]:

# TODO: Se prepara y se parametriza las columnas para geneción de reporte alfanumérico
estandarizacion_columnas_devuXterrenos = ['codigo',
        'radicado_padre',
        'radicado_hijo',
        'ficha',
        'componente_responsable',
        'actividad_actual_tramite',
        'con_plano_cartografico',
        'usuario_inserta_plano',
        'area_ha_cmt12',
        'id_ui',
        'nombre_ui',
        'meta_hito'
        ]

df_devueltosXterrenos = df_devueltosXterrenos[estandarizacion_columnas_devuXterrenos]

# TODO: Se actualiza el reporte de rendimientos
with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_devueltosXterrenos.to_excel(writer, sheet_name=NOMBRE_HOJA, index=False)
